In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
import os

In [2]:
df = pd.read_csv("data/bbb_reindexed_24-4.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/bbb_reindexed_24-4.csv'

In [7]:
df.head()
df.columns

Index(['ball_id', 'match_id', 'match_date', 'venue', 'innings', 'bat_team',
       'bowl_team', 'over', 'ball', 'batter', 'batter_name', 'bowler',
       'bowler_name', 'non_striker', 'runs_batter', 'runs_extras',
       'runs_total', 'bat_team_player_1', 'bat_team_player_2',
       'bat_team_player_3', 'bat_team_player_4', 'bat_team_player_5',
       'bat_team_player_6', 'bat_team_player_7', 'bat_team_player_8',
       'bat_team_player_9', 'bat_team_player_10', 'bat_team_player_11',
       'bowl_team_player_1', 'bowl_team_player_2', 'bowl_team_player_3',
       'bowl_team_player_4', 'bowl_team_player_5', 'bowl_team_player_6',
       'bowl_team_player_7', 'bowl_team_player_8', 'bowl_team_player_9',
       'bowl_team_player_10', 'bowl_team_player_11', 'wicket_type',
       'player_out', 'batter_total_runs', 'batter_balls_faced',
       'bowler_total_runs', 'bowler_balls_bowled', 'team_total_runs', 'rr',
       'target', 'remaining_balls', 'rrr'],
      dtype='object')

In [15]:
date_column = "match_date"        # Replace with actual column name
stadium_column = "venue"  # Replace with actual column name

unique_matches = df[[date_column, stadium_column]].drop_duplicates()

# Convert to a DataFrame
matches_df = pd.DataFrame(unique_matches)
# print(matches_df)

print(matches_df.head())
print(matches_df.info())

      match_date                        venue
0     2002-12-29          McLean Park, Napier
581   2003-01-01   Jade Stadium, Christchurch
1013  2003-01-04      Davies Park, Queenstown
1453  2003-01-08  Westpac Stadium, Wellington
1992  2003-01-09        Sydney Cricket Ground
<class 'pandas.core.frame.DataFrame'>
Index: 2914 entries, 0 to 1547266
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   match_date  2914 non-null   object
 1   venue       2914 non-null   object
dtypes: object(2)
memory usage: 68.3+ KB
None


In [28]:
def get_coordinates(stadium_name):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': stadium_name,
        'format': 'json',
        'limit': 1
    }
    headers = {
        "User-Agent": "MyCricketApp/1.0 (contact@example.com)"  # Replace with your email
    }

    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 403:
        print(f"403 Forbidden: IP might be blocked for: {stadium_name}")
        return None, None
    elif response.status_code != 200:
        print(f"Error {response.status_code} for stadium: {stadium_name}")
        return None, None

    try:
        data = response.json()
        if isinstance(data, list) and len(data) > 0:
            return float(data[0]['lat']), float(data[0]['lon'])
        else:
            print(f"No coordinates found for: {stadium_name}")
            return None, None
    except requests.exceptions.JSONDecodeError:
        print(f"Invalid JSON for: {stadium_name}")
        return None, None
    finally:
        time.sleep(1)  # To avoid rate limit

# Extract unique stadiums
stadium_column = "venue"
unique_stadiums = df[stadium_column].drop_duplicates().reset_index(drop=True)

# Fetch coordinates
coords = unique_stadiums.apply(get_coordinates)

# Convert coordinates into DataFrame
coords_df = pd.DataFrame(coords.tolist(), columns=['lat', 'long'])
unique_stadiums_df = pd.concat([unique_stadiums, coords_df], axis=1)

# Save missing stadiums
missing_stadiums = unique_stadiums_df[unique_stadiums_df['lat'].isnull()]
missing_stadiums.to_csv("data/missing_stadiums_15-4.csv", index=False)

# Save valid stadiums with coordinates
stadiums_with_coords = unique_stadiums_df.dropna(subset=['lat', 'long'])
stadiums_with_coords.to_csv("data/stadiums_with_coordinates_15-4.csv", index=False)

# Merge into main DataFrame
df = df.merge(stadiums_with_coords, on=stadium_column, how="left")

# Save updated match file (optional)
df.to_csv("updated_matches_with_coords.csv", index=False)

print("Coordinates added.")
print(" 'missing_stadiums.csv' and 'stadiums_with_coordinates.csv' saved.")

No coordinates found for: Jade Stadium, Christchurch
No coordinates found for: Davies Park, Queenstown
No coordinates found for: Goodyear Park, Bloemfontein
No coordinates found for: New Wanderers Stadium, Johannesburg
No coordinates found for: North West Cricket Stadium, Potchefstroom
No coordinates found for: Boland Bank Park, Paarl
No coordinates found for: City Oval, Pietermaritzburg
No coordinates found for: Gymkhana Club Ground, Nairobi
No coordinates found for: Sharjah Cricket Association Stadium
No coordinates found for: Bangabandhu National Stadium, Dhaka
No coordinates found for: Rangiri Dambulla International Stadium
No coordinates found for: Queen's Park (New), St George's, Grenada
No coordinates found for: Arnos Vale Ground, Kingstown, St Vincent
No coordinates found for: The Royal & Sun Alliance County Ground, Bristol
No coordinates found for: The Rose Bowl, Southampton
No coordinates found for: Bundaberg Rum Stadium, Cairns
No coordinates found for: Lal Bahadur Shastri S

In [ ]:
# df.to_csv("final_matches.csv", index=False)


In [29]:
# Ensure these columns are in your DataFrame
required_columns = ['match_id', 'match_date', 'venue', 'lat', 'long']
# assert all(col in df.columns for col in required_columns), "Missing required columns in df"

# Extract one row per match with location info
match_location_df = (
    df[required_columns]
    .drop_duplicates(subset=['match_id'])  # one row per match
    .sort_values(by='match_date')
    .reset_index(drop=True)
)

# Save to CSV
match_location_df.to_csv("data/match_location_data_15-4.csv", index=False)

print("✅ Saved unique match-location data to 'data/match_location_data_15-4.csv'")


✅ Saved unique match-location data to 'data/match_location_data_15-4.csv'


In [ ]:
# Load the first 500 rows
location_df = pd.read_csv("data/match_location_data_15-4.csv").iloc[2000:]

print(f"⛅ Fetching weather for {len(location_df)} matches...")

# Function to fetch historical weather from Open-Meteo
def get_weather(lat, lon, date):
    base_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": date,
        "end_date": date,
        "hourly": "temperature_2m,relative_humidity_2m,dew_point_2m,cloud_cover,wind_speed_10m",
        "timezone": "auto"
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if "hourly" in data and len(data["hourly"]["temperature_2m"]) > 0:
                return {
                    "temperature_2m": data["hourly"]["temperature_2m"][0],
                    "relative_humidity_2m": data["hourly"]["relative_humidity_2m"][0],
                    "dew_point_2m": data["hourly"]["dew_point_2m"][0],
                    "cloud_cover": data["hourly"]["cloud_cover"][0],
                    "wind_speed_10m": data["hourly"]["wind_speed_10m"][0]
                }
            else:
                print(f"No weather data: {lat}, {lon}, {date}")
                return None
        else:
            print(f"API error {response.status_code} for {lat}, {lon}, {date}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    finally:
        time.sleep(0.4)

# Gather weather info
weather_metrics = []

for _, row in location_df.iterrows():
    lat, lon = row["lat"], row["long"]
    date = pd.to_datetime(row["match_date"]).date().isoformat()

    if pd.notnull(lat) and pd.notnull(lon):
        weather_info = get_weather(lat, lon, date)
        if weather_info:
            weather_metrics.append(weather_info)
        else:
            weather_metrics.append({
                "temperature_2m": None,
                "relative_humidity_2m": None,
                "dew_point_2m": None,
                "cloud_cover": None,
                "wind_speed_10m": None
            })
    else:
        weather_metrics.append({
            "temperature_2m": None,
            "relative_humidity_2m": None,
            "dew_point_2m": None,
            "cloud_cover": None,
            "wind_speed_10m": None
        })

# Merge with location data
weather_df = pd.concat([location_df.reset_index(drop=True), pd.DataFrame(weather_metrics)], axis=1)

# Save test part
existing_df = pd.read_csv("data/match_location_weather_15-4_part1.csv")

# Add the new weather data
combined_df = pd.concat([existing_df, weather_df], ignore_index=True)

# Save the combined dataset back to the same file
combined_df.to_csv("data/match_location_weather_15-4_part1.csv", index=False)

print("Saved: 'data/match_location_weather_15-4_part1.csv'")


⛅ Fetching weather for 1000 matches...
Saved: 'data/match_location_weather_15-4_part1.csv'
